In [2]:
import pandas as pd
import numpy as np

# Lire les fichiers de données
train_df = pd.read_csv('data/train.csv')
client_df = pd.read_csv('data/client.csv')
electricity_prices_df = pd.read_csv('data/electricity_prices.csv')
gas_prices_df = pd.read_csv('data/gas_prices.csv')

In [4]:
train_df_sample = train_df.sample(10000)


In [5]:
train_df_sample

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
326183,9,1,1,123.455,1,2021-12-18 19:00:00,108,326183,36
1207443,7,1,1,742.728,1,2022-09-21 16:00:00,385,1207443,29
1592231,13,0,1,12.884,1,2023-01-17 23:00:00,503,1592231,50
1554174,13,0,1,0.000,0,2023-01-06 03:00:00,492,1554174,50
681565,2,0,1,3.184,1,2022-04-10 13:00:00,221,681565,62
...,...,...,...,...,...,...,...,...,...
1205970,7,1,3,0.109,0,2022-09-21 05:00:00,385,1205970,30
1410516,4,1,1,0.000,0,2022-11-22 17:00:00,447,1410516,66
190308,3,1,1,0.000,0,2021-11-03 21:00:00,63,190308,13
1721982,11,1,1,106.070,0,2023-02-27 12:00:00,544,1721982,46


In [7]:
electricity_prices_df['forecast_date'] = pd.to_datetime(electricity_prices_df['forecast_date'])


In [8]:
electricity_prices_df

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1
...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637


In [9]:
electricity_prices_df['date_str'] = electricity_prices_df['forecast_date'].apply(lambda x: str(x))
electricity_prices_df['date'] = electricity_prices_df['forecast_date'].apply(lambda x: x.strftime('%d-%m-%y'))

In [10]:
electricity_prices_df

,forecast_date,euros_per_mwh,origin_date,data_block_id,date_str,date
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,2021-09-01 00:00:00,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,2021-09-01 01:00:00,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,2021-09-01 02:00:00,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,2021-09-01 03:00:00,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,2021-09-01 04:00:00,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,2023-05-30 19:00:00,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,2023-05-30 20:00:00,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,2023-05-30 21:00:00,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,2023-05-30 22:00:00,30-05-23


In [12]:
train_df_sample['price_elec'] = -1

for unique_date in train_df_sample['datetime'].unique():
    df_elec_filtered = electricity_prices_df[electricity_prices_df['origin_date'] == unique_date]
    if len(df_elec_filtered) != 0:
        if len(df_elec_filtered) == 1:
            elec_price = df_elec_filtered['euros_per_mwh'].values[0]
        else:
            elec_price = df_elec_filtered.iloc[0]['euros_per_mwh']

        train_df_sample.loc[train_df_sample['datetime'] == unique_date, 'price_elec'] = elec_price


/var/folders/bx/5bjglg_13clbm14l0wlx716m0000gn/T/ipykernel_20106/2263906415.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '173.73' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df_sample.loc[train_df['datetime'] == unique_date, 'price_elec'] = elec_price


KeyboardInterrupt: 